In [1]:
!pip install dm-acme
!pip install dm-acme[reverb]
!pip install dm-acme[tf]
!pip install gym

You should consider upgrading via the '/home/pjotr/projects/poc/tutorials/datascience/venv/bin/python3.8 -m pip install --upgrade pip' command.
zsh:1: no matches found: dm-acme[reverb]
zsh:1: no matches found: dm-acme[tf]
You should consider upgrading via the '/home/pjotr/projects/poc/tutorials/datascience/venv/bin/python3.8 -m pip install --upgrade pip' command.


In [1]:
environment_library = 'gym'

In [6]:
import IPython

from acme import environment_loop
from acme import specs
from acme import wrappers
from acme.agents.tf import d4pg
from acme.tf import networks
from acme.tf import utils as tf2_utils
from acme.utils import loggers
import numpy as np
import sonnet as snt
import gym
import time


environment = gym.make('MountainCarContinuous-v0')
environment = wrappers.GymWrapper(environment)  # To dm_env interface.

# Make sure the environment outputs single-precision floats.
environment = wrappers.SinglePrecisionWrapper(environment)

# Grab the spec of the environment.
environment_spec = specs.make_environment_spec(environment)

#@title Build agent networks

# Get total number of action dimensions from action spec.
num_dimensions = np.prod(environment_spec.actions.shape, dtype=int)

# Create the shared observation network; here simply a state-less operation.
observation_network = tf2_utils.batch_concat

# Create the deterministic policy network.
policy_network = snt.Sequential([
    networks.LayerNormMLP((256, 256, 256), activate_final=True),
    networks.NearZeroInitializedLinear(num_dimensions),
    networks.TanhToSpec(environment_spec.actions),
])

# Create the distributional critic network.
critic_network = snt.Sequential([
    # The multiplexer concatenates the observations/actions.
    networks.CriticMultiplexer(),
    networks.LayerNormMLP((512, 512, 256), activate_final=True),
    networks.DiscreteValuedHead(vmin=-150., vmax=150., num_atoms=51),
])

# Create a logger for the agent and environment loop.
agent_logger = loggers.TerminalLogger(label='agent', time_delta=10.)
env_loop_logger = loggers.TerminalLogger(label='env_loop', time_delta=10.)

# Create the D4PG agent.
agent = d4pg.D4PG(
    environment_spec=environment_spec,
    policy_network=policy_network,
    critic_network=critic_network,
    observation_network=observation_network,
    sigma=1.0,
    logger=agent_logger,
    checkpoint=False
)

# Create an loop connecting this agent to the environment created above.
env_loop = environment_loop.EnvironmentLoop(
    environment, agent, logger=env_loop_logger)

In [9]:
env_loop.run(num_episodes=100)

[Agent] Critic Loss = 3.374 | Policy Loss = 0.007 | Steps = 1125 | Walltime = 134.393
[Env Loop] Episode Length = 999 | Episode Return = -57.16551971435547 | Episodes = 11 | Steps = 10989 | Steps Per Second = 267.319
[Agent] Critic Loss = 3.224 | Policy Loss = 0.004 | Steps = 1459 | Walltime = 144.411
[Env Loop] Episode Length = 999 | Episode Return = -51.51047134399414 | Episodes = 14 | Steps = 13986 | Steps Per Second = 258.536
[Agent] Critic Loss = 3.058 | Policy Loss = 0.001 | Steps = 1779 | Walltime = 154.437
[Env Loop] Episode Length = 999 | Episode Return = -61.6729850769043 | Episodes = 17 | Steps = 16983 | Steps Per Second = 240.101
[Agent] Critic Loss = 2.912 | Policy Loss = 0.007 | Steps = 2078 | Walltime = 164.461
[Agent] Critic Loss = 2.767 | Policy Loss = 0.000 | Steps = 2371 | Walltime = 174.489
[Env Loop] Episode Length = 999 | Episode Return = -61.63372039794922 | Episodes = 20 | Steps = 19980 | Steps Per Second = 232.572
[Agent] Critic Loss = 2.650 | Policy Loss = 0.0

[Agent] Critic Loss = 1.937 | Policy Loss = 0.276 | Steps = 10821 | Walltime = 535.261
[Env Loop] Episode Length = 174 | Episode Return = 88.50698852539062 | Episodes = 446 | Steps = 87978 | Steps Per Second = 189.055
[Agent] Critic Loss = 1.841 | Policy Loss = 0.275 | Steps = 11062 | Walltime = 545.292
[Env Loop] Episode Length = 167 | Episode Return = 88.38923645019531 | Episodes = 462 | Steps = 89950 | Steps Per Second = 193.673
[Agent] Critic Loss = 1.855 | Policy Loss = 0.276 | Steps = 11297 | Walltime = 555.298
[Env Loop] Episode Length = 110 | Episode Return = 92.47654724121094 | Episodes = 477 | Steps = 91824 | Steps Per Second = 186.733
[Agent] Critic Loss = 1.873 | Policy Loss = 0.333 | Steps = 11529 | Walltime = 565.298
[Env Loop] Episode Length = 112 | Episode Return = 91.98640441894531 | Episodes = 494 | Steps = 93765 | Steps Per Second = 188.566
[Agent] Critic Loss = 1.825 | Policy Loss = 0.290 | Steps = 11769 | Walltime = 575.327
[Env Loop] Episode Length = 139 | Episode

In [10]:
timestep = environment.reset()
action = agent.select_action(timestep.observation)

while not timestep.last():
    environment.render()
    environment.step(action)
    time.sleep(0.0125)


KeyboardInterrupt: 